In [12]:
import requests,re,time,numpy as np,pickle,asyncio,concurrent.futures

In [13]:
# Setup relevant variables
ZIP = '60615'
pickupDate = '10/16/2017'
dropoffDate = '10/23/2017'
TIME = '9:00 AM'
RADIUS = '15'

In [28]:
%run core_functions_debug.ipynb

In [29]:
results = runparser()
print(results)

001 OK      | ('1', 'city', 'enterprise', 'E11586', 'Gg Hyde Park', 'ET', '0.95')
003 OK      | ('3', 'city', 'avis', 'CHIC05', 'Chicago - 18th Street & Wabash Avenue', 'AV', '4.05')
004 OK      | ('4', 'city', 'budget', 'CHIC02', 'Chicago', 'BG', '4.05')
002 OK      | ('2', 'city', 'enterprise', 'E11576', 'Gg South Loop', 'ET', '3.17')
006 NO DATA | ('6', 'city', 'enterprise', 'E115UG', 'Uber Southside', 'ET', '4.20')
005 OK      | ('5', 'city', 'enterprise', 'E115DN', 'Gg South Western', 'ET', '4.19')
007 OK      | ('7', 'city', 'budget', 'CHIC08', 'Chicago', 'BG', '5.11')
008 OK      | ('8', 'city', 'enterprise', 'E11530', 'Gg Stony Island', 'ET', '5.17')
009 OK      | ('9', 'city', 'enterprise', 'E115DA', 'Gg Gresham', 'ET', '5.57')
010 OK      | ('10', 'city', 'avis', 'CHIX01', 'Chicago Union Train Station', 'AV', '5.71')
011 OK      | ('11', 'city', 'budget', 'CHIX01', 'Chicago Union Train Station', 'BG', '5.71')
012 OK      | ('12', 'city', 'enterprise', 'E115CE', 'Chicago Loop'

In [25]:
car1 = 'Economy Car'
car2 = 'Compact Car'
car3 = 'Intermediate Car'
car4 = 'Intermediate SUV'
desiredcar = car2

results2 = dict()
for k,v in results.items():
    #print(v)
    if desiredcar in v:
        results2[k] = v
order = np.array([x[desiredcar] for x in results2.values()]).argsort()
order = np.array([x[desiredcar] for x in results2.values() if desiredcar in x]).argsort()

In [26]:
brands = np.array([x['brand'] for x in results2.values()])[order]
nums = np.array([x for x in results2.keys()])[order]
titles = np.array([x['name'] for x in results2.values()])[order]
prices = np.array([x[desiredcar] for x in results2.values()])[order]
prices2 = np.array([x[car2] for x in results2.values()])[order]
prices3 = np.array([x[car3] for x in results2.values()])[order]
distances = np.array([x['dist'] for x in results2.values()])[order]

print(data['pickupDate'],data['pickupTime'],data['dropoffDate'],data['dropoffTime'],desiredcar,car2,car3)
for k in zip(nums,prices,prices2,prices3,brands,titles,distances):
    #print(k)
    print('{:03d} | {:5.2f} | {:7.2f} | {:7.2f} | {:7.2f} | {:12s} | {}'.format(k[0],float(k[6]),k[1],k[2],k[3],str(k[4]),k[5]))

10/16/2017 9:00 AM 10/23/2017 9:00 AM Compact Car Compact Car Intermediate Car
010 |  5.71 |  194.52 |  194.52 |  193.12 | avis         | Chicago Union Train Station
018 |  6.07 |  197.43 |  197.43 |  201.92 | alamo        | Chicago N Lasalle
005 |  4.19 |  198.88 |  198.88 |  209.77 | enterprise   | Gg South Western
008 |  5.17 |  198.88 |  198.88 |  209.77 | enterprise   | Gg Stony Island
009 |  5.57 |  198.88 |  198.88 |  209.77 | enterprise   | Gg Gresham
017 |  6.05 |  200.32 |  200.32 |  203.05 | budget       | North Loop Budget
011 |  5.71 |  202.12 |  202.12 |  202.24 | budget       | Chicago Union Train Station
004 |  4.05 |  203.48 |  203.48 |  211.17 | budget       | Chicago
007 |  5.11 |  203.48 |  203.48 |  211.17 | budget       | Chicago
015 |  5.93 |  207.32 |  207.32 |  219.43 | enterprise   | Millennium Park
019 |  6.09 |  207.32 |  207.32 |  219.43 | enterprise   | Lake And Franklin
012 |  5.82 |  208.69 |  208.69 |  219.58 | enterprise   | Chicago Loop
013 |  5.84 | 